In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import numpy as np
from scipy import stats as st
import re

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [8]:
train.columns

Index(['id', '賃料', '所在地', 'アクセス', '間取り', '築年数', '方角', '面積', '所在階', 'バス・トイレ',
       'キッチン', '放送・通信', '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間'],
      dtype='object')

In [7]:
train.describe()

,id,賃料
count,31470.000000,3.147000e+04
mean,15735.500000,1.182903e+05
std,9084.750822,7.374880e+04
min,1.000000,2.500000e+04
25%,7868.250000,7.600000e+04
50%,15735.500000,1.020000e+05
75%,23602.750000,1.360000e+05
max,31470.000000,2.500000e+06


In [6]:
train.head()

,id,賃料,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間


In [12]:
for col in train.columns:
    if train[col].isnull().sum() > 0:
        print(col + " " + str(train[col].isnull().sum()))

方角 2749
バス・トイレ 329
キッチン 621
放送・通信 3440
室内設備 82
駐車場 5287
周辺環境 9432
契約期間 7120


In [13]:
print(train["面積"].str.contains("ms").count())

31470


In [14]:
print(train["築年数"].str.contains("月").count())

31470


In [15]:
print(train["面積"].values[0][:-2])

20.01


In [48]:
test = "29年4ヶ月"

year = r"[0-9]+年"
month = r"[0-9]+ヶ月"

In [65]:
m = re.search(month,test)
print(m)
print(m[0])

<re.Match object; span=(3, 6), match='4ヶ月'>
4ヶ月


In [74]:
train.nunique()

id        31470
賃料          871
所在地       11333
アクセス      24172
間取り          30
築年数         746
方角            8
面積         5739
所在階        1032
バス・トイレ      360
キッチン        381
放送・通信       139
室内設備       6220
駐車場        3911
周辺環境      17864
建物構造         10
契約期間         66
dtype: int64

In [73]:
print(train["間取り"].unique())

['1K' '1R' '2LDK' '2DK' '1DK' '1LDK' '3LDK' '3DK' '1LDK+S(納戸)' '4K' '2K'
 '1K+S(納戸)' '4LDK' '3LDK+S(納戸)' '5LDK+S(納戸)' '5LDK' '3K' '4DK'
 '2LDK+S(納戸)' '2DK+S(納戸)' '4LDK+S(納戸)' '5DK' '3DK+S(納戸)' '1DK+S(納戸)' '5K'
 '6LDK' '2K+S(納戸)' '1LK+S(納戸)' '5DK+S(納戸)' '3K+S(納戸)']


In [77]:
temp = train["所在階"][0]

In [81]:
x,y = temp.split("／")

In [82]:
print(x)
print(y)

1階
12階建


In [89]:
train["アクセス"].str.contains("徒歩").count()

31470

In [103]:
p1 = ".+線.+駅.+分"
p2 = ".+線"
p3 = "線.+駅"
p4 = "徒歩.+分"
txt = train["アクセス"][0]

base = re.match(p1,txt)[0]
print(base)
print(re.match(p2,base))
print(re.search(p4,base))

都営三田線	西巣鴨駅	徒歩4分		埼京線	板橋駅	徒歩14分		都電荒川線	西ケ原四丁目駅	徒歩7分
<re.Match object; span=(0, 37), match='都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線'>
<re.Match object; span=(11, 50), match='徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\t西ケ原四丁目駅\t徒歩7分'>


In [104]:
print(train["アクセス"][345].split())

['京急本線', '梅屋敷(東京都)駅', '徒歩5分', '京急空港線', '京急蒲田駅', '徒歩13分', '京浜東北線', '蒲田駅', '徒歩19分']


In [88]:
m[0][3:-1]

'北'

In [95]:
train.groupby("方角").count()

,id,賃料,所在地,アクセス,間取り,築年数,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
方角,,,,,,,,,,,,,,,,
北,1966,1966,1966,1966,1966,1966,1966,1966,1952,1930,1803,1961,1682,1393,1966,1548
北東,1321,1321,1321,1321,1321,1321,1321,1321,1310,1309,1223,1321,1118,980,1321,1031
北西,1326,1326,1326,1326,1326,1326,1326,1326,1323,1317,1228,1325,1122,917,1326,1041
南,8909,8909,8909,8909,8909,8909,8909,8909,8850,8770,7868,8886,7441,6252,8909,6902
南東,3098,3098,3098,3098,3098,3098,3098,3098,3070,3043,2766,3091,2628,2124,3098,2435
南西,3080,3080,3080,3080,3080,3080,3080,3080,3058,3022,2768,3075,2585,2191,3080,2407
東,4995,4995,4995,4995,4995,4995,4995,4995,4953,4927,4531,4989,4247,3608,4995,3952
西,4026,4026,4026,4026,4026,4026,4026,4026,3983,3967,3660,4019,3405,2853,4026,3169


In [113]:
d = {}

for i in range(31470):
    if train["バス・トイレ"][i] != train["バス・トイレ"][i]:
        continue
    for key in train["バス・トイレ"][i].split():
        pat = "／"
        if re.sub(pat,"",key) not in d:
            d[re.sub(pat,"",key)] = 1
        else:
            d[re.sub(pat,"",key)] += 1
print(d)

{'専用バス': 23782, '専用トイレ': 23465, 'バス・トイレ別': 26458, 'シャワー': 28242, '浴室乾燥機': 16337, '': 15533, '温水洗浄便座': 17545, '洗面台独立': 20237, '脱衣所': 12361, '追焚機能': 11989, '共同トイレ': 86, 'バスなし': 141, '共同バス': 34}


In [150]:
d = {}

for i in range(31470):
    if train["キッチン"][i] != train["キッチン"][i]:
        continue
    for key in train["キッチン"][i].split():
        pat = "／"
        tmp = re.sub(pat,"",key)
        if re.match(r"コンロ設置可.*",tmp):
            tmp = "コンロ設置可"
        if tmp not in d:
            d[tmp] = 1
        else:
            d[tmp] += 1
print(d)

{'ガスコンロ': 20369, 'コンロ2口': 14947, 'システムキッチン': 20534, '': 26364, '給湯': 26368, '独立キッチン': 3907, 'コンロ3口': 5393, 'IHコンロ': 4431, 'コンロ1口': 3057, '冷蔵庫あり': 677, 'コンロ設置可': 2102, 'カウンターキッチン': 3724, 'L字キッチン': 309, '電気コンロ': 525, 'コンロ4口以上': 50}


In [152]:
l = {}
ind = 0
for key in d.keys():
    if key != "": 
        l[key] = ind
        ind += 1
print(l)

{'ガスコンロ': 0, 'コンロ2口': 1, 'システムキッチン': 2, '給湯': 3, '独立キッチン': 4, 'コンロ3口': 5, 'IHコンロ': 6, 'コンロ1口': 7, '冷蔵庫あり': 8, 'コンロ設置可': 9, 'カウンターキッチン': 10, 'L字キッチン': 11, '電気コンロ': 12, 'コンロ4口以上': 13}


In [138]:
key = "コンロ設置可（コンロ3口）"
pat = "／"
tmp = re.sub(pat,"",key)
# tmp = re.search(r"\（.+\）",tmp)
# print(tmp)
# print(tmp[0])
tmp = re.sub(r"\（.+\）","",tmp)
print(tmp)

コンロ設置可


In [153]:
d = {}

for i in range(31470):
    if train["室内設備"][i] != train["室内設備"][i]:
        continue
    for key in train["室内設備"][i].split():
        pat = "／"
        if re.sub(pat,"",key) not in d:
            d[re.sub(pat,"",key)] = 1
        else:
            d[re.sub(pat,"",key)] += 1
print(d)

{'エアコン付': 29746, 'シューズボックス': 22947, 'バルコニー': 25112, 'フローリング': 27274, '室内洗濯機置場': 27389, '敷地内ごみ置き場': 16746, '': 24588, 'エレベーター': 16148, '公営水道': 23242, '下水': 23415, '都市ガス': 25533, 'タイル張り': 13677, 'ウォークインクローゼット': 3848, '2面採光': 6636, '24時間換気システム': 10704, '3面採光': 1046, 'ペアガラス': 597, '専用庭': 215, '水道その他': 1685, '冷房': 5179, 'クッションフロア': 394, '床暖房': 1635, 'プロパンガス': 1010, 'ロフト付き': 1241, '出窓': 1314, 'トランクルーム': 504, '汲み取り': 5, 'オール電化': 549, 'ルーフバルコニー': 316, '室外洗濯機置場': 1695, '床下収納': 587, 'バリアフリー': 157, '浄化槽': 10, '防音室': 61, '二重サッシ': 87, '二世帯住宅': 3, 'ガスその他': 16, '洗濯機置場なし': 69, '排水その他': 9, '石油暖房': 1, '地下室': 7, 'ガス暖房': 2, '井戸': 1}


In [154]:
l = {}
ind = 0
for key in d.keys():
    if d[key] > 50 and key != "":
        l[key] = ind
        ind += 1
print(l)

{'エアコン付': 0, 'シューズボックス': 1, 'バルコニー': 2, 'フローリング': 3, '室内洗濯機置場': 4, '敷地内ごみ置き場': 5, 'エレベーター': 6, '公営水道': 7, '下水': 8, '都市ガス': 9, 'タイル張り': 10, 'ウォークインクローゼット': 11, '2面採光': 12, '24時間換気システム': 13, '3面採光': 14, 'ペアガラス': 15, '専用庭': 16, '水道その他': 17, '冷房': 18, 'クッションフロア': 19, '床暖房': 20, 'プロパンガス': 21, 'ロフト付き': 22, '出窓': 23, 'トランクルーム': 24, 'オール電化': 25, 'ルーフバルコニー': 26, '室外洗濯機置場': 27, '床下収納': 28, 'バリアフリー': 29, '防音室': 30, '二重サッシ': 31, '洗濯機置場なし': 32}


In [155]:
d = {}

for i in range(31470):
    if train["放送・通信"][i] != train["放送・通信"][i]:
        continue
    for key in train["放送・通信"][i].split():
        pat = "／"
        if re.sub(pat,"",key) not in d:
            d[re.sub(pat,"",key)] = 1
        else:
            d[re.sub(pat,"",key)] += 1
print(d)

{'インターネット対応': 17193, 'CATV': 13083, 'CSアンテナ': 9291, 'BSアンテナ': 13453, '光ファイバー': 16306, '高速インターネット': 2498, 'インターネット使用料無料': 5404, '有線放送': 72}


In [156]:
l = {}
ind = 0
for key in d.keys():
    if d[key] > 50 and key != "":
        l[key] = ind
        ind += 1
print(l)

{'インターネット対応': 0, 'CATV': 1, 'CSアンテナ': 2, 'BSアンテナ': 3, '光ファイバー': 4, '高速インターネット': 5, 'インターネット使用料無料': 6, '有線放送': 7}


In [167]:
d = {"無":0,"有":0}
place = [r"駐輪場\t",r"駐車場\t",r"バイク置き場\t"]
available = [r"空有",r"[(無)|(近隣)|(空無)]"]
all_pat = []

for p in place:
    for a in available:
        all_pat.append(p+a)
exist = all_pat[::2]
dont = all_pat[1::2]
print(exist)
print(dont)


for i in range(31470):
    if train["駐車場"][i] != train["駐車場"][i]:
        continue
    for pat in exist:
        if re.search(pat,train["駐車場"][i]):
            d["有"]　+= 1
        else:
            d["無"]　+= 1
print(d)

['駐輪場\\t空有', '駐車場\\t空有', 'バイク置き場\\t空有']
['駐輪場\\t[(無)|(近隣)|(空無)]', '駐車場\\t[(無)|(近隣)|(空無)]', 'バイク置き場\\t[(無)|(近隣)|(空無)]']
{'駐輪場\t空有': 16490, '駐輪場\t空': 16505, '駐車場\t無': 8597, 'バイク置き場\t無': 12642, '駐車場\t近': 6627, 'バイク置き場\t空有': 5096, 'バイク置き場\t空': 5143, '駐輪場\t無': 4513, '駐車場\t空有': 5738, '駐車場\t空': 6193, 'バイク置き場\t近': 173, '駐輪場\t近': 2}


In [173]:
d = [0 for i in range(6)]
exist = [r"駐輪場\t空有",r"駐車場\t空有",r"バイク置き場\t空有"]
# available = [r"空有",r"[(無)|(近隣)|(空無)]"]
# all_pat = []

# for p in place:
#     for a in available:
#         all_pat.append(p+a)
# exist = all_pat[::2]
# dont = all_pat[1::2]
# print(exist)
# print(dont)


for i in range(31470):
    if train["駐車場"][i] != train["駐車場"][i]:
        continue
    for j in range(len(exist)):
        if re.search(exist[j],train["駐車場"][i]):
            d[j] += 1
        else:
            d[j+3] += 1
print(d)

[16490, 5738, 5096, 9693, 20445, 21087]


In [174]:
d = {}

for i in range(31470):
    if train["周辺環境"][i] != train["周辺環境"][i]:
        continue
    for key in train["周辺環境"][i].split():
        pat = "／"
        p2 = "【.*】"
        if re.match(p2,key):
            if re.sub(pat,"",key) not in d:
                d[re.sub(pat,"",key)] = 1
            else:
                d[re.sub(pat,"",key)] += 1
print(d)

{'【小学校】': 2655, '【大学】': 2052, '【公園】': 3807, '【飲食店】': 6929, '【スーパー】': 40666, '【コンビニ】': 34944, '【ドラッグストア】': 5594, '【郵便局】': 3478, '【病院】': 3804, '【図書館】': 1299, '【銀行】': 1267, '【学校】': 1551, '【幼稚園・保育園】': 1824, '【総合病院】': 1890, '【デパート】': 691, '【レンタルビデオ】': 502, '【クリーニング】': 146, '【コインパーキング】': 22, '【月極駐車場】': 2}


In [176]:
l = {}
ind = 0
for key in d.keys():
    if d[key] > 50 and key != "":
        l[key] = ind
        ind += 1
print(l)

{'【小学校】': 0, '【大学】': 1, '【公園】': 2, '【飲食店】': 3, '【スーパー】': 4, '【コンビニ】': 5, '【ドラッグストア】': 6, '【郵便局】': 7, '【病院】': 8, '【図書館】': 9, '【銀行】': 10, '【学校】': 11, '【幼稚園・保育園】': 12, '【総合病院】': 13, '【デパート】': 14, '【レンタルビデオ】': 15, '【クリーニング】': 16}
